# 6. FEATURE ENGINEERING

# 6.1. BASELINE MODEL 

In [2]:
# Chargement des librairies
import pandas as pd

In [3]:
#  astuce pour afficher toutes les lignes sans les ...
pd.set_option('display.max_row', 20)
#  astuce pour afficher toutes les colonnes dans les head()
pd.set_option('display.max_column', 20)

In [4]:
# Charger un fichier csv

from datetime import datetime
dateparse = lambda dates: [pd.datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in dates]

kickstarter_path = 'C:/Users/PC Maison/4-KAGGLE/KAGGLE_DEV/KAGGLE_COURS_6-FEATURE_ENGINEERING/kickstarter-projects/input/'
kickstarter = pd.read_csv(kickstarter_path + 'ks-projects-201612.csv',
                 parse_dates=['launched ','deadline '],
                 date_parser=dateparse,
                 infer_datetime_format=True,
                 encoding = 'ISO-8859-1')

C:\Users\Outils\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-4-6c6145219996>:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  dateparse = lambda dates: [pd.datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in dates]


ValueError: time data '2' does not match format '%Y-%m-%d %H:%M:%S'

## 1.1. COURS

In [49]:
kickstarter.columns

Index(['ID ', 'name ', 'category ', 'main_category ', 'currency ', 'deadline ',
       'goal ', 'launched ', 'pledged ', 'state ', 'backers ', 'country ',
       'usd pledged ', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16'],
      dtype='object')

In [50]:
# suppression de l'espace après le nom des colonnes
ks = kickstarter.rename(columns={'ID ': 'ID',
                        'name ': 'name',
                        'category ': 'category',
                        'main_category ': 'main_category',
                        'currency ': 'currency',
                        'deadline ': 'deadline',
                        'goal ': 'goal',
                        'launched ': 'launched',
                        'pledged ': 'pledged',
                        'state ': 'state',
                        'backers ': 'backers',
                        'country ': 'country',
                        'usd pledged ': 'usd pledged'})
ks.columns

Index(['ID', 'name', 'category', 'main_category', 'currency', 'deadline',
       'goal', 'launched', 'pledged', 'state', 'backers', 'country',
       'usd pledged', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16'],
      dtype='object')

In [51]:
# supprimer les colonnes 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
# 'Unnamed: 16'
ks.drop(['Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16'], 
          axis=1, inplace=True)
ks.columns

Index(['ID', 'name', 'category', 'main_category', 'currency', 'deadline',
       'goal', 'launched', 'pledged', 'state', 'backers', 'country',
       'usd pledged'],
      dtype='object')

In [52]:
print('Unique values in `state` column:', list(ks.state.unique()))

Unique values in `state` column: ['failed', 'canceled', 'successful', 'live', 'undefined', 'suspended', '4262', '3984', '525', '0', '3035', '3273', '590', '23924', '12306', '13675', '2', '4690', '4488.01', '687', '450', '16185', '80', '1495', '20', '14309.36', '2011-06-04 01:17:48', '8542', '3363', '950', '1800.5', '352', '1320', '18', '2161', '2016-06-28 21:36:59', '331', '17861', '1', '2630', '1085', '3711', '103', '8609.6', '312', '11044', '5600.65', '630', '1010', '178', '2530', '3671', '65', '5', '4451', '16', '2000', '377', '125', '6169', '90', '765', '218.5', '1850', '166', '2107', '63', '10', '3300', '306', '13', '400', '7703', '650', '800', '4287', '1056', '5940', '45', '1827', '10890.45', '245', '276', '200', '30', '6', '797', '6126', '175', '12201', '3605.13', '1200', '325', '2632', '7271', '46', '2016-02-18 21:40:23', '465', '9430.8', '2308', '310', '7470', '2759', '9857.8', '6453', '6189', '6093', '51', '860', '6442', '390', '1595', '1763.47', '130', '11315.5', '25', '3141

## PREPARE TARGET VALUE

In [53]:
# First we'll convert the state column into a target we can use in a model. Data cleaning isn't the current focus, so we'll simplify this example by:
# Dropping projects that are "live"
# Counting "successful" states as outcome = 1
# Combining every other state as outcome = 0

In [54]:
# Drop live projects
ks = ks.query('state != "live"')


# Add outcome column, "successful" == 1, others are 0
ks = ks.assign(outcome=(ks['state'] == 'successful').astype(int))
ks

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,outcome
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0,0
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220,0
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1,0
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283,0
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323745,999976400,ChknTruk Nationwide Charity Drive 2014 (Canceled),Documentary,Film & Video,USD,2014-10-17 02:35:30,50000,2014-09-17 02:35:30,25,canceled,1,US,25,0
323746,999977640,The Tribe,Narrative Film,Film & Video,USD,2011-07-19 03:35:14,1500,2011-06-22 03:35:14,155,failed,5,US,155,0
323747,999986353,Walls of Remedy- New lesbian Romantic Comedy f...,Narrative Film,Film & Video,USD,2010-08-16 05:59:00,15000,2010-07-01 19:40:30,20,failed,1,US,20,0
323748,999987933,BioDefense Education Kit,Technology,Technology,USD,2016-02-13 02:00:00,15000,2016-01-13 18:13:53,200,failed,6,US,200,0


## CONVERT TIMESTAMPS

In [37]:
# Next, we convert the launched feature into categorical features we can
# use in a model. 
# Since we loaded the columns as timestamp data, we access date and time 
# values through the .dt attribute on the timestamp column.

In [64]:
# ks['launched'] = pd.to_datetime(ks['launched'], format='%Y-%m-%d %H:%M:%S')
# ks['launched'] = ks['launched'].dt.date 
ks['launched'].head()

0    2015-08-11 12:12:28
1    2013-01-12 00:20:50
2    2012-03-17 03:24:11
3    2015-07-04 08:35:03
4    2016-02-26 13:38:27
Name: launched, dtype: object

In [38]:
ks = ks.assign(hour=ks.launched.dt.hour,
               day=ks.launched.dt.day,
               month=ks.launched.dt.month,
               year=ks.launched.dt.year)

AttributeError: Can only use .dt accessor with datetimelike values

## PREPARE CATEGORICAL FEATURES

In [ ]:
# Now for the categorical variables -- category, currency, and country 
# -- we'll need to convert them into integers so our model can use the data
# For this we'll use scikit-learn's LabelEncoder. This assigns an integer 
# to each value of the categorical feature.

In [ ]:
from sklearn.preprocessing import LabelEncoder

cat_features = ['category', 'currency', 'country']
encoder = LabelEncoder()

# Apply the label encoder to each column
encoded = ks[cat_features].apply(encoder.fit_transform)

In [ ]:
# We collect all of these features in a new dataframe that we can use to
# train a model.

In [ ]:
# Since ks and encoded have the same index and I can easily join them
data = ks[['goal', 'hour', 'day', 'month', 'year', 'outcome']].join(encoded)
data.head()

## CREATION TRAIN SET - VALIDE SET ET TEST SET

In [ ]:
# We need to create data sets for training, validation, and testing. 
# We'll use a fairly simple approach and split the data using slices. 
# We'll use 10% of the data as a validation set, 10% for testing, and the 
# other 80% for training.

In [ ]:
valid_fraction = 0.1
valid_size = int(len(data) * valid_fraction)

train = data[:-2 * valid_size]
valid = data[-2 * valid_size:-valid_size]
test = data[-valid_size:]

## TRAIN MODEL

In [ ]:
# For this course we'll be using a LightGBM model. 
# This is a tree-based model that typically provides the best performance, 
# even compared to XGBoost. It's also relatively fast to train.
# We won't do hyperparameter optimization because that isn't the goal of 
# this course. 
# So, our models won't be the absolute best performance you can get. 
# But you'll still see model performance improve as we do feature 
# engineering.

In [ ]:
import lightgbm as lgb

feature_cols = train.columns.drop('outcome')

dtrain = lgb.Dataset(train[feature_cols], label=train['outcome'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['outcome'])

param = {'num_leaves': 64, 'objective': 'binary'}
param['metric'] = 'auc'
num_round = 1000
bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], 
                early_stopping_rounds=10, verbose_eval=False)

## MAKE PREDICTIONS & EVALUATE MODEL

In [ ]:
from sklearn import metrics
ypred = bst.predict(test[feature_cols])
score = metrics.roc_auc_score(test['outcome'], ypred)

print(f"Test AUC score: {score}")
# Test AUC score: 0.747615303004287

## 1.2. EXERCICES

In [51]:
# Charger un fichier csv
click_data_path = 'C:/Users/PC Maison/4-KAGGLE/KAGGLE_DEV/KAGGLE_COURS_6-FEATURE_ENGINEERING/feature-engineering-data/input/'
click_data = pd.read_csv(click_data_path + 'train_sample.csv', 
                          parse_dates=['click_time'])

In [52]:
click_data.shape

(2300561, 8)

In [53]:
click_data.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,89489,3,1,13,379,2017-11-06 15:13:23,NaN,0
1,204158,35,1,13,21,2017-11-06 15:41:07,2017-11-07 08:17:19,1
2,3437,6,1,13,459,2017-11-06 15:42:32,NaN,0
3,167543,3,1,13,379,2017-11-06 15:56:17,NaN,0
4,147509,3,1,13,379,2017-11-06 15:57:01,NaN,0


## 1.2.1. CONSTRUCT FEATURES FROM TIMESTAMP

In [54]:
# Notice that the click_data DataFrame has a 'click_time' column with 
# timestamp data.
# Use this column to create features for the coresponding day, hour, minute
# and second.
# Store these as new integer columns day, hour, minute, and second in a new
# DataFrame clicks.

In [55]:
# Add new columns for timestamp features day, hour, minute, and second
clicks = click_data.copy()
clicks['day'] = clicks['click_time'].dt.day.astype('uint8')
# Fill in the rest
clicks['hour'] = clicks['click_time'].dt.hour.astype('uint8')
clicks['minute'] = clicks['click_time'].dt.minute.astype('uint8')
clicks['second'] = clicks['click_time'].dt.second.astype('uint8')

## 1.2.2. LABEL ENCODING

In [56]:
# For each of the categorical features ['ip', 'app', 'device', 'os', 
# 'channel'], use scikit-learn's LabelEncoder to create new features in 
# the clicks DataFrame. The new column names should be the original column 
# name with '_labels' appended, like ip_labels.

In [57]:
from sklearn import preprocessing

cat_features = ['ip', 'app', 'device', 'os', 'channel']

# Create new columns in clicks using preprocessing.LabelEncoder()
label_encoder = preprocessing.LabelEncoder()

for feature in cat_features:
        encoded = label_encoder.fit_transform(clicks[feature])
        clicks[feature + '_labels'] = encoded

In [58]:
clicks.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second,ip_labels,app_labels,device_labels,os_labels,channel_labels
0,89489,3,1,13,379,2017-11-06 15:13:23,NaN,0,6,15,13,23,27226,3,1,13,120
1,204158,35,1,13,21,2017-11-06 15:41:07,2017-11-07 08:17:19,1,6,15,41,7,110007,35,1,13,10
2,3437,6,1,13,459,2017-11-06 15:42:32,NaN,0,6,15,42,32,1047,6,1,13,157
3,167543,3,1,13,379,2017-11-06 15:56:17,NaN,0,6,15,56,17,76270,3,1,13,120
4,147509,3,1,13,379,2017-11-06 15:57:01,NaN,0,6,15,57,1,57862,3,1,13,120


## TRAIN TEST SPLITS WITH TIME SERIES DATA

In [59]:
feature_cols = ['day', 'hour', 'minute', 'second', 
                'ip_labels', 'app_labels', 'device_labels',
                'os_labels', 'channel_labels']

valid_fraction = 0.1
clicks_srt = clicks.sort_values('click_time')
valid_rows = int(len(clicks_srt) * valid_fraction)
train = clicks_srt[:-valid_rows * 2]
# valid size == test size, last two sections of the data
valid = clicks_srt[-valid_rows * 2:-valid_rows]
test = clicks_srt[-valid_rows:]

## TRAIN WITH MODEL LightGBM

In [60]:
import lightgbm as lgb

dtrain = lgb.Dataset(train[feature_cols], label=train['is_attributed'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['is_attributed'])
dtest = lgb.Dataset(test[feature_cols], label=test['is_attributed'])

param = {'num_leaves': 64, 'objective': 'binary'}
param['metric'] = 'auc'
num_round = 1000
bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], early_stopping_rounds=10)

[1]	valid_0's auc: 0.948979
Training until validation scores don't improve for 10 rounds
[2]	valid_0's auc: 0.949235
[3]	valid_0's auc: 0.950126
[4]	valid_0's auc: 0.950072
[5]	valid_0's auc: 0.950536
[6]	valid_0's auc: 0.950943
[7]	valid_0's auc: 0.951453
[8]	valid_0's auc: 0.951518
[9]	valid_0's auc: 0.952385
[10]	valid_0's auc: 0.952434
[11]	valid_0's auc: 0.952465
[12]	valid_0's auc: 0.952638
[13]	valid_0's auc: 0.95266
[14]	valid_0's auc: 0.952766
[15]	valid_0's auc: 0.953203
[16]	valid_0's auc: 0.953503
[17]	valid_0's auc: 0.953793
[18]	valid_0's auc: 0.953966
[19]	valid_0's auc: 0.954184
[20]	valid_0's auc: 0.9543
[21]	valid_0's auc: 0.954305
[22]	valid_0's auc: 0.954536
[23]	valid_0's auc: 0.954748
[24]	valid_0's auc: 0.955142
[25]	valid_0's auc: 0.955493
[26]	valid_0's auc: 0.955611
[27]	valid_0's auc: 0.955708
[28]	valid_0's auc: 0.955795
[29]	valid_0's auc: 0.956172
[30]	valid_0's auc: 0.95623
[31]	valid_0's auc: 0.956477
[32]	valid_0's auc: 0.956606
[33]	valid_0's auc: 0.95

## EVALUATE MODEL

In [61]:
from sklearn import metrics

ypred = bst.predict(test[feature_cols])
score = metrics.roc_auc_score(test['is_attributed'], ypred)
print(f"Test score: {score}")

Test score: 0.9726727334566094
